## Übung zu Unsicherheiten

#### 1) Messwerte mit Unsicherheiten
Gegeben sind die Messwerte mit Messunsicherheiten eines Zylinders aus unbekanntem Material. Ermittelt werden soll die Dichte $\rho$ des Zylinders und die zugehörigen Unsicherheiten $\Delta\rho$.

In [1]:
# load essential libraries
import numpy as np 


m = 2670 #g
D = 52.1 #mm
H = 160. #mm

dm = 2   #g
dD = 0.1 #mm
dH = 0.5 #mm

#### 2) Erwartungswert $\overline{\rho}$ der Dichte

In [3]:
# expectation value
rho = 4 * m / (np.pi * D**2 * H)
print('rho [g/mm^3]', rho)
print('rho [g/cm^3]', rho * 1e3)

rho [g/mm^3] 0.00782755180785807
rho [g/cm^3] 7.827551807858071


#### 3) Messunsicherheit $\Delta \rho$ der Dichte 

In [5]:
# uncertainty
drho = rho * ((dm / m)**2 + (2 * dD / D)**2 + (dH / H)**2)**0.5
print('drho [g/mm^3]', drho)
print('drho [g/cm^3]', drho * 1e3)

drho [g/mm^3] 3.91869545965466e-05
drho [g/cm^3] 0.0391869545965466


#### 4) Relative Unsicherheit der Dichte $\frac{\Delta \rho}{\overline{\rho}}$

In [7]:
# relative uncertainty
rel = drho / rho
print('relative [.]', rel)
print('relative [%]', rel * 100)

relative [.] 0.0050062849225995356
relative [%] 0.5006284922599535


#### 5) Monte Carlo Betrachtung der Unsicherheiten $\left(\Delta m = 2g,~\Delta D = 0.1 mm,~\Delta H = 0.5mm\right)$

Es wird nun "numerisch" 1 Mio. mal gemessen. Welchen Wert erhalten wir dann für die Unsicherheit $\Delta\rho_{MonteCarlo}$?

In [9]:
# Monte Carlo

import numpy as np
np.random.seed(seed=2)

# number of measurements
N = 1000000

# values for m, D, H after N measurements
m_MC = np.random.normal(m, dm, N)
D_MC = np.random.normal(D, dD, N)
H_MC = np.random.normal(H, dH, N)

# calculated possible N values for rho 
rho_MC = 4 * m_MC / (np.pi * D_MC**2 * H_MC)

# mean and std of rho
rho_MC_mean = np.mean(rho_MC)
rho_MC_std = np.std(rho_MC)

print('Monte Carlo: rho [g/mm^3]:', rho_MC_mean, '+-', rho_MC_std)
print('')
print('Monte Carlo: rho [g/cm^3]:', rho_MC_mean * 1e3, '+-', rho_MC_std * 1e3)
print('Gauss Error: rho [g/cm^3]:', rho * 1e3, '+-', drho * 1e3)

Monte Carlo: rho [g/mm^3]: 0.007827747464370816 +- 3.919448355004817e-05

Monte Carlo: rho [g/cm^3]: 7.827747464370816 +- 0.03919448355004817
Gauss Error: rho [g/cm^3]: 7.827551807858071 +- 0.0391869545965466


#### 5) Monte Carlo Betrachtung der Unsicherheiten $\left(\Delta m = [2;20]g,~\Delta D = [0.1;10]mm,~\Delta H = [0.5;10]mm\right)$

Es wird nun "numerisch" 1 Mio. mal gemessen. Welchen Wert erhalten wir dann für die Unsicherheit $\Delta\rho_{MonteCarlo}$ wenn es größere Ausgangsunsicherheiten $\Delta m$, $\Delta D$ oder $\Delta H$ gibt?

In [11]:
# Monte Carlo widget

import numpy as np
np.random.seed(seed=2)

# for general plotting
import matplotlib.pyplot as plt
%matplotlib inline

# interactive plots
from ipywidgets import interactive
import ipywidgets as widgets

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 200

def update(dm = widgets.IntSlider(min=2., max=20., step=6, value=2.,     description='dm [g]:'), 
           dD = widgets.FloatSlider(min=0.1, max=10., step=2, value=0.1, description='dD [mm]:'), 
           dH = widgets.FloatSlider(min=0.5, max=10., step=2, value=0.5, description='dH [mm]:')):
    
    # Gaussian Error Propagation
    rho_Gauss = 4 * m / (np.pi * D**2 * H)
    drho_Gauss = rho_Gauss * ((dm / m)**2 + (2 * dD / D)**2 + (dH / H)**2)**0.5

    
    # Monte Carlo Error Propagation
    N = 1000000

    m_MC = np.random.normal(m, dm, N)
    D_MC = np.random.normal(D, dD, N)
    H_MC = np.random.normal(H, dH, N)
    
    rho_MC = 4 * m_MC / (np.pi * D_MC**2 * H_MC)
    
    # mean and standard deviation
    rho_MC_mean = np.mean(rho_MC)
    rho_MC_std = np.std(rho_MC)
    
    # results
    #print('Monte Carlo: rho [g/mm^3]:', '%1.5f'%rho_MC_mean, '+-', '%1.5f'%rho_MC_std)
    #print('')
    print('Monte Carlo: rho [g/cm^3]:', '%1.2f'%(rho_MC_mean * 1e3), '+-', '%1.2f'%(rho_MC_std * 1e3))
    print('Gauss Error: rho [g/cm^3]:', '%1.2f'%(rho_Gauss * 1e3), '+-', '%1.2f'%(drho_Gauss * 1e3))
    
    fig = plt.figure(figsize=(8,4))
    
    # Diagram: histogram, bins = 500
    h = plt.hist(rho_MC * 1e3, bins=500, density=True, label='MC: dm=' + str(dm) + 'g, dD=' + str(dD) + 'mm, dH=' + str(dH) + 'mm')   
    
    # for plot of gauss function using dF and F_mean from above
    xmin = np.percentile(rho_MC * 1e3, 0., axis=0)
    xmax = np.percentile(rho_MC * 1e3, 99.9, axis=0)
    xfine = np.linspace(xmin, xmax, 100)
    gauss = 1. / (2 * np.pi * (drho_Gauss*1e3)**2)**0.5 * np.exp(-(xfine - rho_Gauss*1e3)**2/(2 * (drho_Gauss*1e3) **2))
    plt.plot(xfine, gauss, 'r-', alpha=0.5, label=r'Gauss $\frac{1}{\sqrt{2 \pi \sigma^2}}\ \exp{-\frac{(x-\overline{\rho})^2}{2\sigma^2}}$')
    
    
    plt.xlabel(r'Dichte $\rho (g/cm^3)$')
    plt.ylabel('genormte Anzahl')
    plt.legend(loc='best',prop={'size': 12})
    plt.xlim(xmin, xmax)
    
    plt.show()



#### 6) Interaktives Diagramm

Frage 1: Wie verändert sich das Histogramm für große Werte in $\Delta m$, $\Delta D$, $\Delta H$?

Frage 2: Wie verhält es sich mit Kombinationen?

In [13]:
#Frage 1: Wie verändert sich das Histogramm für große Werte in dm, dD, dH?
#Frage 2: Wie verhält es sich mit Kombinationen?

interactive_plot = interactive(update)
interactive_plot

interactive(children=(IntSlider(value=2, description='dm [g]:', max=20, min=2, step=6), FloatSlider(value=0.1,…